In [10]:
%profile "cloudbend"

%glue_version "3.0"
%iam_role "arn:aws:iam::898546127587:role/GlueSessions"
%idle_timeout 60

%worker_type "G.1X"
%number_of_workers 5

%extra_jars "s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar"

Previous profile: None
Setting new profile to: cloudbend
Setting Glue version to: 3.0
Current iam_role is None
iam_role has been set to arn:aws:iam::898546127587:role/GlueSessions.
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Extra jars to be included:
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-clien

In [12]:
%%configure
{
    "conf": "spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
}

You are already connected to a glueetl session bfefe1ee-8362-4b3c-b00e-fecd888a7466.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


The following configurations have been updated: {'conf': 'spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

In [3]:
catalog = "iceberg"
bucket = "pyicebergbenchmark-icebergbucket89dd3fa6-1qhkzajoxgpzo"
prefix = "iceberg"

iceberg_conf = [
    (f"spark.sql.catalog.{catalog}", "org.apache.iceberg.spark.SparkCatalog"),
    (f"spark.sql.catalog.{catalog}.warehouse", f"s3://{bucket}/{prefix}/{catalog}/"),
    (f"spark.sql.catalog.{catalog}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog"),
    (f"spark.sql.catalog.{catalog}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"),
]

for k, v in iceberg_conf:
    spark.conf.set(k, v)

In [44]:
import boto3
from pyspark.sql.functions import col

s3 = boto3.client("s3")

# list all files
response = s3.list_objects_v2(Bucket="nyc-tlc", Prefix="trip data/")
paths = [f"s3://nyc-tlc/{x['Key']}" for x in response["Contents"]]

rdds = []
for path in paths:
    df = (
        spark.read
            .parquet(path)
            .withColumn("trip_type", col("trip_type").cast("string")) # bigints & doubles in source data
            .withColumn("rate_code", col("rate_code").cast("string")) # bigints & strings in source data
            .withColumn("improvement_surcharge", col("improvement_surcharge").cast("double")) # ints & doubles in source data
            .withColumn("congestion_surcharge", col("congestion_surcharge").cast("double")) # ints & doubles in source data
            .withColumn("ehail_fee", col("ehail_fee").cast("double")) # ints & doubles in source data
            .withColumn("VendorID", col("VendorID").cast("string")) # ints & bigints in source data
            .withColumn("SR_Flag", col("SR_Flag").cast("string")) # ints & doubles in source data
            .withColumn("PUlocationID", col("PUlocationID").cast("string")) # bigints & doubles in source data
    )

    rdds.append(df.rdd)

df = spark.sparkContext.union(rdds).toDF()

df.count()

AnalysisException: cannot resolve '`trip_type`' given input columns: [Affiliated_base_number, DOlocationID, PUlocationID, SR_Flag, dispatching_base_num, dropOff_datetime, pickup_datetime];
'Project [dispatching_base_num#278, pickup_datetime#279, dropOff_datetime#280, PUlocationID#281, DOlocationID#282, SR_Flag#283, Affiliated_base_number#284, cast('trip_type as string) AS trip_type#292]
+- Relation[dispatching_base_num#278,pickup_datetime#279,dropOff_datetime#280,PUlocationID#281,DOlocationID#282,SR_Flag#283,Affiliated_base_number#284] parquet



In [55]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)


In [56]:
df.show()

+--------------------+-------------------+-------------------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|Affiliated_base_number|
+--------------------+-------------------+-------------------+----------------------+
|              B00001|2017-06-01 00:04:00|2017-06-07 01:55:36|                      |
|              B00001|2017-06-01 00:47:00|2017-06-09 01:25:40|                      |
|              B00001|2017-06-01 00:07:00|2017-06-15 00:52:17|                      |
|              B00001|2017-06-01 00:30:00|2017-06-03 02:18:52|                      |
|              B00001|2017-06-01 00:45:00|2017-06-10 01:49:42|                      |
|              B00001|2017-06-01 00:25:00|2017-06-19 00:58:48|                      |
|              B00013|2017-06-01 00:30:19|2017-06-01 00:33:34|                B00013|
|              B00013|2017-06-01 00:26:12|2017-06-02 00:27:45|                B00013|
|              B00014|2017-06-01 00:54:03|2017-06-01 0

In [57]:
df.createOrReplaceTempView("nyc_taxi")

In [49]:
%%sql

SELECT COUNT(DISTINCT dispatching_base_num)
FROM nyc_taxi;

+------------------------------------+
|count(DISTINCT dispatching_base_num)|
+------------------------------------+
|                                1333|
+------------------------------------+


In [58]:
%%sql

CREATE DATABASE IF NOT EXISTS iceberg.benchmark;

++
||
++
++


In [59]:
%%sql

CREATE TABLE iceberg.benchmark.nyc_taxi
USING iceberg
PARTITIONED BY (dispatching_base_num)
TBLPROPERTIES (
    'write.parquet.compression-codec' = 'snappy',
    'write.metadata.metrics.default' = 'full'
)
AS
SELECT *
FROM nyc_taxi
ORDER BY pickup_datetime;

Py4JJavaError: An error occurred while calling o69.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 195 in stage 60.0 failed 4 times, most recent failure: Lost task 195.3 in stage 60.0 (TID 4458) (172.34.78.30 executor 3): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainBinaryDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scal

In [14]:
%stop_session

Stopping session: bfefe1ee-8362-4b3c-b00e-fecd888a7466
Stopped session.
